## Some tasks can be broken down into independent subtasks that can be executed simultaneously. This pattern takes advantage of parallel execution to improve efficiency while maintaining the benefits of structured workflows. For example, analyzing multiple documents or processing different aspects of a single input concurrently (like code review).


In [ ]:
import { generateText, generateObject } from 'ai'
import { z } from 'zod'
import { createCompatibleOpenAI } from '../../init.ts'

const openai = createCompatibleOpenAI()

const securityReviewSchema = z.object({
  vulnerabilities: z.array(z.string()),
  riskLevel: z.enum(['low', 'medium', 'high']),
  suggestions: z.array(z.string())
})

const performanceReviewSchema = z.object({
  issues: z.array(z.string()),
  impact: z.enum(['low', 'medium', 'high']),
  optimizations: z.array(z.string())
})

const maintainabilityReviewSchema = z.object({
  concerns: z.array(z.string()),
  qualityScore: z.number().min(1).max(10),
  recommendations: z.array(z.string())
})

async function parallelCodeReview(code: string) {
  const model = openai('gpt-4o-mini')

  const [securityReview, performanceReview, maintainabilityReview] = await Promise.all([
    generateObject<z.infer<typeof securityReviewSchema>>({
      model,
      schema: securityReviewSchema,
			system: 'You are an expert in code security. Focus on identifying security vulnerabilities, injection risks, and authentication issues.',
      prompt: `Review this code: ${code}`
    }),
    generateObject<z.infer<typeof performanceReviewSchema>>({
      model,
      schema: performanceReviewSchema,
			system: 'You are an expert in code performance. Focus on identifying performance issues, bottlenecks, memory leaks, and optimization opportunities.',
      prompt: `Review this code: ${code}`
    }),
    generateObject<z.infer<typeof maintainabilityReviewSchema>>({
      model,
      schema: maintainabilityReviewSchema,
			system: 'You are an expert in code maintainability. Focus on identifying code readability, code structure, and adherence to best practices.',
      prompt: `Review this code: ${code}`
    })
  ])

	const reviews = [
    { ...securityReview.object, type: 'security' },
    { ...performanceReview.object, type: 'performance' },
    { ...maintainabilityReview.object, type: 'maintainability' },
  ];

  // Aggregate results using another model instance
  const { text: summary } = await generateText({
    model,
    system: 'You are a technical lead summarizing multiple code reviews.',
    prompt: `Synthesize these code review results into a concise summary with key actions:
    ${JSON.stringify(reviews, null, 2)}`,
  });

  return { reviews, summary }
}


const code = `
const sum = (a, b) => a + b;
const result = sum(1, 2);
console.log(result);
`

const { reviews, summary } = await parallelCodeReview(code)
console.log(reviews)
console.log(summary)